## AOD1B data product

In [1]:
# Imports
from gfeatpy.gravity import AOD1B, AOD1BType, GravityAnomaly, DateTime
from gfeatpy import plot as plt

import numpy as np
from matplotlib.animation import FuncAnimation, PillowWriter
import warnings, matplotlib

warnings.filterwarnings("ignore", category=matplotlib.MatplotlibDeprecationWarning)

In [2]:
# Read data and create array of SH objects
sh_list = []
date = []

for day in np.arange(1, 32):
    # Load static gravity field daily files
    aod1b = AOD1B()
    aod1b.load(f"aod1b/AOD1B_2025-01-{day:02d}_X_07.asc")
    # Loop over hours
    for h in np.arange(0, 24, 3):
        datetime = DateTime(2025, 1, day, h)   # 2025-01-01 03:00:00
        sh_list.append(aod1b.get(datetime, AOD1BType.GLO))  # Get SH class and append
        date.append(f"2025-Jan-{day} {h:02d}:00")

In [ ]:
%%capture

# Define map resolution
n_lon = 360
n_lat = 180
functional = GravityAnomaly()

# Define maximum values
zlim = 0.05
levels = np.linspace(-zlim, zlim, 101)

# Plot dummy map
ax = plt.get_basemap_axes()
[lon, lat, z] = sh_list[0].synthesis(n_lon, n_lat, functional)
contour = ax.contourf(lon, lat, z, levels=levels, alpha=0.6, cmap='seismic', antialiased=True)
cbar = plt.plt.colorbar(contour, ax=ax, shrink=0.5)
plt.adjust_colorbar_ticks(cbar)
cbar.set_label("$\Delta g$ [mGal]")

# Get ax object
ax = plt.gca()
fig = plt.plt.gcf()

def update(frame):
    global contour, cbar
    # Remove previous map
    for c in contour.collections:
        c.remove()
    # Plot new map
    [lon, lat, z] = sh_list[frame].synthesis(n_lon, n_lat, functional)
    contour = ax.contourf(lon, lat, z, levels=levels, alpha=0.6, cmap='seismic', antialiased=True)
    ax.set_title(date[frame])
    # Return values
    return contour.collections


# Create and save animation
ani = FuncAnimation(fig, update, frames=len(sh_list), blit=True)
ani.save("aod1b-2025-01.gif", writer=PillowWriter(fps=10))

Synthesis progress: 180/180s progress: 39/180s progress: 39/180s: 148/180s progress: 39/180is progress: 40/180s progress: 39/180s progress: 39/180 progress: 76/180s progress: 39/180s progress: 39/180s progress: 39/180s progress: 39/180

![AOD1B Jan-2025](./aod1b-2025-01.gif)